In [1]:
!pip install mygene statannotations scrublet scanpy scvelo decoupler matplotlib_venn goatools gseapy scperturb biomart PyComplexHeatmap statsmodels omnipath git+https://github.com/saezlab/pypath.git --quiet

In [2]:
import subprocess
import os
import sys
import matplotlib.backends.backend_pdf
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns

from pathlib import Path

# Jupyter stuff
from tqdm.notebook import tqdm
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

# Custom functions
sys.path.insert(1, '../')
from utils import *

# scperturb package
sys.path.insert(1, '../package/src/')
from scperturb import *

from pathlib import Path
figure_path = Path('../figures/')

/tmp/ipykernel_1788/1400239515.py:17: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')
path = TEMPDIR / 'ZapateroTape2023'

In [4]:
adata = sc.read_h5ad(path  / 'merged_filtered_trellis_adata_v4.h5ad')

In [5]:
adata

AnnData object with n_obs × n_vars = 31572 × 30600
    obs: 'sub_lib_id', 'sum', 'detected', 'total', 'sample_id', 'well_indexes', 'barcode_seq', 'mapped_reads', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'condition', 'pdo', 'cell_type_condition', 'log10_total_counts', 'outlier_mt', 'outlier_total', 'lowq_total', 'n_genes', 'doublet_score', 'predicted_doublet', 'dataset', 'leidenr0.1', 'cell_type', 'doublet_outliers', 'pdo_sample_id', 'pdo_replicate'
    var: 'gene_ids', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'cell_type_condition_colors', 'hvg', 'leiden', 'leidenr0.1_colors', 'log1p', 'neighbors', 'pca', 'pdo_colors', 'pdo_sample_id_colors', 'predicted_doublet_colors'
    obsm: 'X_pca', 'X_phate'
    varm: 'PCs'
    layers: 'counts', 'reads'
    obsp: 'connectivities', 'distances'

In [11]:
adata.X = adata.layers['counts'].copy()
del adata.layers['counts']

In [13]:
adata.obs

,sub_lib_id,sum,detected,total,sample_id,well_indexes,barcode_seq,mapped_reads,n_genes_by_counts,total_counts,...,lowq_total,n_genes,doublet_score,predicted_doublet,dataset,leidenr0.1,cell_type,doublet_outliers,pdo_sample_id,pdo_replicate
17_2_2_sl1_pdo_21,exp013_p21_s4,2625.0,1436,2625.0,PDO_CAF_1,17_2_2,AAACATCGAAACATCGCGCGACTA,23608.0,1436,2625.0,...,False,1436,0.010961,False,pdo_21,0,PDO,False,pdo21_PDO_CAF_1,pdo21_1
5_2_2_sl1_pdo_21,exp013_p21_s4,1590.0,952,1590.0,PDO_1,5_2_2,AAACATCGAAACATCGTGGTATAC,14500.0,952,1590.0,...,False,952,0.068054,False,pdo_21,0,PDO,False,pdo21_PDO_1,pdo21_1
32_13_2_sl1_pdo_21,exp013_p21_s4,3962.0,1583,3962.0,PDO_CAF_2,32_13_2,AAACATCGAACAACCATTCCGATC,25972.0,1583,3962.0,...,False,1583,0.023308,False,pdo_21,0,PDO,False,pdo21_PDO_CAF_2,pdo21_2
21_14_2_sl1_pdo_21,exp013_p21_s4,4015.0,2288,4015.0,PDO_2,21_14_2,AAACATCGAACCGAGAACGCCGGC,38949.0,2288,4015.0,...,False,2288,0.024257,False,pdo_21,0,PDO,False,pdo21_PDO_2,pdo21_2
46_14_2_sl1_pdo_21,exp013_p21_s4,8457.0,3507,8457.0,PDO_CAF_3,46_14_2,AAACATCGAACCGAGACTATTTCA,69217.0,3507,8457.0,...,False,3507,0.056944,False,pdo_21,0,PDO,False,pdo21_PDO_CAF_3,pdo21_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12_64_68_sl2_pdo_27,exp013_p27_s5,5215.0,2609,5215.0,PDO_CAF_1,12_64_68,TTCACGCATGAAGAGACCGTTCTA,69619.0,2609,5215.0,...,False,2609,0.029029,False,pdo_27,2,PDO,False,pdo27_PDO_CAF_1,pdo27_1
38_65_68_sl2_pdo_27,exp013_p27_s5,3250.0,1435,3250.0,CAF_3,38_65_68,TTCACGCATGGAACAACGCTTAAA,41136.0,1435,3250.0,...,False,1435,0.021353,False,pdo_27,1,Fibroblast,False,pdo27_CAF_3,pdo27_3
13_67_68_sl2_pdo_27,exp013_p27_s5,6883.0,2541,6883.0,PDO_CAF_1,13_67_68,TTCACGCATGGTGGTATGGCGCGC,81594.0,2541,6883.0,...,False,2541,0.030501,False,pdo_27,1,Fibroblast,False,pdo27_PDO_CAF_1,pdo27_1
16_68_68_sl2_pdo_27,exp013_p27_s5,6275.0,2507,6275.0,PDO_CAF_1,16_68_68,TTCACGCATTCACGCAAATTTCTC,68240.0,2507,6275.0,...,False,2507,0.024540,False,pdo_27,2,PDO,False,pdo27_PDO_CAF_1,pdo27_1


In [14]:
adata.obs[['condition', 'pdo', 'cell_type_condition']]

,condition,pdo,cell_type_condition
17_2_2_sl1_pdo_21,Coculture,pdo21,PDO_Coculture
5_2_2_sl1_pdo_21,PDO,pdo21,PDO_PDO
32_13_2_sl1_pdo_21,Coculture,pdo21,PDO_Coculture
21_14_2_sl1_pdo_21,PDO,pdo21,PDO_PDO
46_14_2_sl1_pdo_21,Coculture,pdo21,PDO_Coculture
...,...,...,...
12_64_68_sl2_pdo_27,Coculture,pdo27,PDO_Coculture
38_65_68_sl2_pdo_27,Fibroblast,pdo27,Fibroblast_Fibroblast
13_67_68_sl2_pdo_27,Coculture,pdo27,Fibroblast_Coculture
16_68_68_sl2_pdo_27,Coculture,pdo27,PDO_Coculture
